In [74]:
import numpy as np
import math
import numpy.matlib
import sys

In [75]:
class Signal:
    fc = 28                                # carrier in GHz
    c = 0.3                                # speed of light [m/ns]
    T = 200                                # number of observations
    P = 1                                  # transmit power mW
    N0dBmHz = -174                         # dBm/Hz
    N0 = pow(10, 0.1 * N0dBmHz) * 1e9 # noise PSD  [mW/GHz] (290 Kelvin * Boltzmann constant in W/Hz)
    BW = 1e-3                              # Bandwidth GHz
    NFdB = 8                               # receiver noise figure [dB]
    NF = pow(10, 0.1 * NFdB)
    EsN0 = 2 * P / (NF * N0 * BW)
    sigma2 = NF * N0 * BW/2
    Lambda = c / fc                        # wavelength
    plot = 0                               # 1 = show plots
    sims = 5                               # number of Monte Carlo simulations


In [76]:
class RIS:

        M = 50 * 50                                                         # RIS elements 
        Delta = Signal.Lambda/2                                             # RIS element spacing
        Location = np.zeros((3, 1))                                         # location of RIS in XY plane
        Antenna = np.array([0, 0, -1 * Signal.Lambda]).reshape((3, 1))      # RIS antenna location
        Orientation = 0
        bits = 1000
        beam_type = "random"    # 'position', 'direction', 'random'
        print(Antenna)

        

[[ 0.        ]
 [ 0.        ]
 [-0.01071429]]


In [77]:
def computeRISChannel(source, IRS, Signal, regime):
            M = IRS.M
            fc = Signal.fc
            c = Signal.c
            spacing = IRS.Delta
            RIS = IRS.Location
            Lambda = c / fc             # Wavelenght(m)
            A = pow(2, spacing)         # basic element area
            a = math.sqrt(A)            # basic element size

            phi = math.atan2(source[1], source[0])
            theta = math.acos(source[2] / np.linalg.norm(source))
            k = 2 * np.pi / Lambda * np.array([math.cos(phi)*math.sin(theta), math.sin(phi)*math.sin(theta), math.cos(theta)]).reshape((3, 1))
            gain = np.zeros((M, 1))
            phase_rot = np.zeros((M, 1))

            iix = np.linspace(0, math.sqrt(M) - 1)
            iiy = np.linspace(0, math.sqrt(M) - 1)
            iix = a * (iix - math.sqrt(M) / 2)
            iiy = a * (iiy - math.sqrt(M) / 2)

            locations = np.zeros((3, M))

            ###### verify below this
            coord = np.matlib.repmat(np.linspace(1, math.sqrt(M), dtype=int), int(math.sqrt(M)), 1)
            x_temp = iix[coord - 1]
            y_temp = iiy[np.transpose(coord) - 1]
            xyz_mat = np.array([np.transpose(x_temp), np.transpose(y_temp)])
           
            ###### verify above this


            locations[0:2,:] = xyz_mat
            d=np.linalg.norm(locations-source)
            d0=np.linalg.norm(source)

            correction = 1- pow(2, math.sin(theta)) * pow(2, math.sin(phi))

            if regime == "CM1":
            #         phase_rot=mod(-k'*locations,2*pi);
            #         phase_rot=phase_rot';
            #         gain=((sqrt(cos(theta)*correction)*a)/(sqrt(4*pi)*norm(source-RIS)))*exp(-1i*phase_rot);
                pass
            elif regime == "CM2":
            #         phase_rot=mod(2*pi*(d-d0)/lambda,2*pi);        
            #         phase_rot=phase_rot';
            #         gain=((sqrt(cos(theta)*correction)*a)/(sqrt(4*pi)*norm(source-RIS)))*exp(-1i*phase_rot);
                pass
            elif regime == "CM3":
                  for m in range(M):          
                    el_m =locations[:,m]
                    x_m=el_m(0)
                    y_m=el_m(1)       
                    setX=[a/2+x_m-source(0),a/2-x_m+source(0)]             
                    setY=[a/2+y_m-source(1),a/2-y_m+source(1)]                                             
                    d=abs(source(2))                            #according to the paper, the Z-coordinate is d
                    TMP=np.zeros(length(setX),length(setY))
                    dm=np.linalg.norm(source-el_m)                       #distance between BS and m-th element
                    for ix in range(len(setX)):
                        for iy in range(len(setY)):                      
                            a2=(setX(ix)*setY(iy))/pow(2,d)                        
                            b21=3*(pow(2,setY(iy))/pow(2,d))+3                        
                            b22=math.sqrt(((pow(2,setX(ix))+pow(2,setY(iy)))/pow(2,d))+1)                       
                            TMP[ix,iy]=(a2/(b21*b22))+((2/3) * math.atan2(a2,b22))                       
                               
                    powerval=((1/(4*np.pi))*np.sum(np.sum(TMP)))                #power of the m-th element                         
                    phase_rot[m]=2*np.pi*(dm-d0)/Lambda
                    gain[m]=math.sqrt(powerval) * np.exp(-1j*phase_rot(m))      #complex channel gain                               
            pass


            return (gain,phase_rot,locations)                          #RIS rotation around the Y axis 


In [80]:
gain,phase_rot,locations = computeRISChannel(RIS.Antenna,RIS,Signal,'CM3') # response from RIS to antenna
RIS.b = gain
print(RIS.b)


ValueError: could not broadcast input array from shape (2,50,50) into shape (2,2500)

In [66]:
class UE:
    covariance = np.eye(3, dtype=int)               # UE a priori covariance
    distances = np.array([0.15, 1, 1.5, 2, 3, 4, 6, 8, 10, 12, 15], dtype=float)
    e = np.array([1, 1, 1])
    e_unit = e / np.linalg.norm(e)
    rhoRange = []
    phiRange = []
    thetaRange = []
    print(e_unit)
    print(distances)
    print(len(distances))
    print(distances.shape)
    


[0.57735027 0.57735027 0.57735027]
[ 0.15  1.    1.5   2.    3.    4.    6.    8.   10.   12.   15.  ]
11
(11,)


In [67]:
def quantizePhases(phases_in, nbits):
    if nbits > 5:
        phases_out = phases_in
    else:
        delta = np.pi / nbits
        phases_out = np.floor(phases_in / delta + 0.5) * delta
    
    return phases_out

In [68]:
ab=quantizePhases(np.angle(RIS.b),RIS.bits)
print(ab)



AttributeError: type object 'RIS' has no attribute 'b'

In [38]:
Omega_b=np.diag(np.exp(-1j*ab))  
print(Omega_b)
bnew=RIS.b.transpose()*Omega_b
errorStat=np.zeros([len(UE.distances),Signal.sims])
RMSE=np.zeros([1,len(UE.distances)])
print(errorStat)
print(RMSE)


NameError: name 'ab' is not defined

In [ ]:
#def getBeams(UE,RIS,signal,beamType):
   # switcher = {
       # random:   
       #     phases=rand(signal.T,RIS.M)*2*pi
        #    phases=quantizePhases(phases,RIS.bits)          
          #  Beams=exp(1j*phases),
      #  direction:
        
    #}
    
def showContours(Signal,UE,RIS):
    x_grid = np.linespace(-2,2,50)
    z_grid = np.linespace(0.001,4,50)
    Omega_b = np.diag(np.conj(np.divide(b,(abs(b)))))
    bnew = b.transpose() * Omega_b
    PEB = np.zeros(length(x_grid),length(z_grid))
    SNRr = np.zeros(length(x_grid),length(z_grid))
    SNRd = np.zeros(length(x_grid),length(z_grid))
    SNRp = np.zeros(length(x_grid),length(z_grid))
    
    UE.Location=0.1*[1,1,1]       #assume beams are pointing here        
    UE.rhoRange=np.norm(UE1.Location)
    UE.phiRange=np.atan2(UE1.Location(2),UE1.Location(1))    #between 0 and 2pi
    UE.thetaRange=acos(UE1.Location(3)/norm(UE1.Location)) #between 0 and pi (for us pi/2, since Z>0)  
    UE.mean=UE.Location 
    UE.covariance=UE.covariance      
    BeamsR=getBeams(UE1,RIS,signal,'random')
    BeamsD=getBeams(UE1,RIS,signal,'direction')       
    BeamsP=getBeams(UE1,RIS,signal,'position')       
    UE.Location = np.zeros(3,1)
    
    

In [ ]:
#gain, phase_rot, locations = RIS.b
#gain = np.angle(gain)
#phase_rot = np.angle(phase_rot)
#locations = np.angle(locations)
#RIS.b = (gain, phase_rot, locations)
#ab=quantizePhases(RIS.b,RIS.bits)

